In [1]:
# Shubham Gajera Shopify Summer InternShip 2022 
import csv
import pandas as pd

In [2]:
data_df=pd.read_csv("Shopify_Data.csv")
data_df.head()

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
0,1,53,746,224,2,cash,2017-03-13 12:36:56
1,2,92,925,90,1,cash,2017-03-03 17:38:52
2,3,44,861,144,1,cash,2017-03-14 4:23:56
3,4,18,935,156,1,credit_card,2017-03-26 12:43:37
4,5,18,883,156,1,credit_card,2017-03-01 4:35:11


In [3]:
# Identify data info
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   order_id        5000 non-null   int64 
 1   shop_id         5000 non-null   int64 
 2   user_id         5000 non-null   int64 
 3   order_amount    5000 non-null   int64 
 4   total_items     5000 non-null   int64 
 5   payment_method  5000 non-null   object
 6   created_at      5000 non-null   object
dtypes: int64(5), object(2)
memory usage: 273.6+ KB


In [4]:
data_df.groupby(['total_items']).count()

,order_id,shop_id,user_id,order_amount,payment_method,created_at
total_items,,,,,,
1,1830,1830,1830,1830,1830,1830
2,1832,1832,1832,1832,1832,1832
3,941,941,941,941,941,941
4,293,293,293,293,293,293
5,77,77,77,77,77,77
6,9,9,9,9,9,9
8,1,1,1,1,1,1
2000,17,17,17,17,17,17


* There are total 17 transection which are unusual and bought 2000 items at once. It's likely that these excessively large transactions are driving up the AOV. At first glance we can assume that this transactions are outliers.

let's inspect more. 

In [5]:
unusual_data=data_df[data_df['total_items']==2000]
unusual_data

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
15,16,42,607,704000,2000,credit_card,2017-03-07 4:00:00
60,61,42,607,704000,2000,credit_card,2017-03-04 4:00:00
520,521,42,607,704000,2000,credit_card,2017-03-02 4:00:00
1104,1105,42,607,704000,2000,credit_card,2017-03-24 4:00:00
1362,1363,42,607,704000,2000,credit_card,2017-03-15 4:00:00
1436,1437,42,607,704000,2000,credit_card,2017-03-11 4:00:00
1562,1563,42,607,704000,2000,credit_card,2017-03-19 4:00:00
1602,1603,42,607,704000,2000,credit_card,2017-03-17 4:00:00
2153,2154,42,607,704000,2000,credit_card,2017-03-12 4:00:00
2297,2298,42,607,704000,2000,credit_card,2017-03-07 4:00:00


* **After carefull inspection I found that all order made by single users with same order_amount and we can also see that the time of this order executed is different day same time. So it must be some automated order request by user to buy same shoes again and again. So we should check if it is anauthorised order or there is some issues while adding this order data to our database.**

In [7]:
# Incorrect calculation explained as a rather than carefull data analysis, analyzer include extreme points in calculation which lead to large sum of average order value 
total_order_amount= data_df['order_amount'].sum()
total_count = data_df['order_id'].count()
print("Total order amount is", total_order_amount)
print("Total number of item sold is",total_count)
AOV = total_order_amount/total_count
print("Wrong AOV ",AOV)

Total order amount is 15725640
Total number of item sold is 5000
Wrong AOV  3145.128


far as I think because of some unsual extream data points we should calculate our AOV according to Median, which give more better overview of each transactions.And this matric not affect by extreame cases.

In [8]:
better_AOV=data_df["order_amount"].median()
better_AOV

284.0

# Answers:

## Question 1.a
### Think about what could be going wrong with our calculation. Think about a better way to evaluate this data.
* When using the AOV, calculation done with average order amount without taking into account the number of pairs of shoes purchased in each transaction.
* there are some transactions that are extream/outliers/error:The 17 indentical transactions that included 2000 purchases each are driving up the AOV.

## Question 1.B: 
### What metric would you report for this dataset?
* As far as I think because of some unsual extream data points we should calculate our AOV according to Median, which give more better overview of each transactions and will not affect with extreame cases or outliers.

## Question 1.C:
### What is its value?
* I would say 284 is more reasonable value for AOV compare to 3145.128 which lead by extrme points/outliers.

## Question 2a.
### How many orders were shipped by Speedy Express in total?

* Answer: 54

* SELECT COUNT(*)
FROM Orders AS o, Shippers AS s
WHERE o.ShipperId = s.ShipperId AND ShipperName = "Speedy Express";

## Question 2b.
### What is the last name of the employee with the most orders?
* Answer: Handel
(10 orders)

* SELECT c.CustomerName, COUNT(*) AS Count
FROM Orders AS o, Customers AS c
WHERE o.CustomerID = c.CustomerID
GROUP BY o.CustomerID
ORDER BY Count DESC
LIMIT 1;

## Question 2c.
### What product was ordered the most by customers in Germany?
* Answer: Boston Crab Meat
(ProductID: 40, TotalQuantity: 160)

* SELECT p.ProductID, p.ProductName, SUM(Quantity) AS TotalQuantity
FROM Orders AS o, OrderDetails AS od, Customers AS c, Products AS p
WHERE c.Country = "Germany" AND od.OrderID = o.OrderID AND od.ProductID = p.ProductID AND c.CustomerID = o.CustomerID
GROUP BY p.ProductID
ORDER BY TotalQuantity DESC
LIMIT 1;